In [0]:
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
# !mkdir -p drive
# !google-drive-ocamlfuse drive

In [0]:
# !cp drive/images.zip ./
# !unzip images.zip

In [6]:
import tensorflow as tf
import numpy as np
import os, pdb
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes, layer_count 
n_classes = 100
layer_count = 0

'''
LOG_DIR = '/tmp/log2'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

#======================Only needed in colab===================================
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
#======================Only needed in colab===================================
'''

'\nLOG_DIR = \'/tmp/log2\'\nget_ipython().system_raw(\n    \'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &\'\n    .format(LOG_DIR)\n)\n\n#======================Only needed in colab===================================\n!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip\n!unzip -o ngrok-stable-linux-amd64.zip\nget_ipython().system_raw(\'./ngrok http 6006 &\')\n!curl -s http://localhost:4040/api/tunnels | python3 -c     "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"\n#======================Only needed in colab===================================\n'

In [0]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file, training_img_dir):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filename = training_img_dir+filename
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=256):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    
    #example = tf.image.decode_png(file_contents, channels=3, name="dataset_image") # png fo rlfw
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn

def setup_inputs(sess, filenames, training_img_dir, image_size=256, crop_size=224, isTest=False, batch_size=64):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames, training_img_dir)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    # Crop and other random augmentations
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_hue(image,0.05)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
    
        

    image = tf.random_crop(image, [crop_size, crop_size, 3])
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')

    tf.train.start_queue_runners(sess=sess)

    return image, y, fn, len(label_list)

In [0]:
def initializer(in_filters, out_filters,name='wb', ks=3):
    W = tf.get_variable(name+"W", [3,3, in_filters,out_filters], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable(name+"B", [out_filters], initializer=tf.truncated_normal_initializer())
    return W, b

def activation(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, w, b, s, p):
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = l_input+b

    return l_input

def max_pool(name, l_input, k, s):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='VALID', name=name)

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, momentum = 0.997, training=isTraining, name="bn"+name)

In [0]:

def residual_block(in_x, in_filters, out_filters, stride, tst, name):
    in_channels = in_x.get_shape().as_list()[3]
    # first convolution layer
    global layer_count
    ##"How to have skip connection?"
    w1, b1 = initializer(in_filters, in_filters, name=name+'wb1')
    x = conv2d(name+'r1', in_x, w1, b1, stride, "SAME")
    x = activation(x)   
    layer_count+=1
    print("Layer [%d] " % (layer_count) , x.get_shape().as_list())
    
    # second convolution layer
    w2, b2 = initializer(in_filters, out_filters, name=name+'wb2')
    
    x = conv2d(name+'r2', x, w2, b2, stride, "SAME")
    x = batchnorm(x, tst, name=name+"bn1-2")
    x = activation(x)
    layer_count+=1
    if in_filters != out_filters:
        difference = out_filters - in_filters
        left_pad = difference // 2
        right_pad = difference - left_pad
        identity = tf.pad(in_x, [[0, 0], [0, 0], [0, 0], [left_pad, right_pad]])
        return identity + x
        return identity + x 
    else:
       
        return in_x + x
def ResNet(_X, tst):
    global n_classes
    w1 = tf.get_variable("firstW", [7,7,3, 64], initializer=tf.truncated_normal_initializer())
    b1 = tf.get_variable("firstB", [64], initializer=tf.truncated_normal_initializer())
    
    x = conv2d('conv1', _X, w1, b1, 3, "VALID")
    x = batchnorm(x, tst, name='sbn')
    x= tf.nn.relu(x)
    
    filters_num = [64,128,256,512]
    block_num = [3,4,6,3]
    strides=[1,1,1,1]
    l_cnt = 1
    for i in range(len(filters_num)):
      for j in range(block_num[i]):
          x = residual_block(x, filters_num[i], filters_num[i], strides[i], tst, 'RB%d_%d'%(i,j))
          print('[L-%d] Build %dth residual block %d with %d channels' % (l_cnt,i, j, filters_num[i]))
          l_cnt +=1
          if ((j==block_num[i]-1) & (i<len(filters_num)-1)):
            x = batchnorm(x, tst, name='RB_bn%d_%d'%(i,j))
            w1, b1 = initializer(filters_num[i], filters_num[i+1], name='RB_pool%d_%d'%(i,j))
            x = conv2d('RB_pool%d_%d'%(i,j), x, w1, b1, 2, "VALID")
            x = activation(x)
            print('[L-%d] Build %dth connection layer %d from %d to %d channels' % (l_cnt, i, j, filters_num[i], filters_num[i+1]))
            l_cnt +=1

    wo, bo=initializer(filters_num[-1], n_classes, name='final_wb')
    x = conv2d('final', x, wo, bo, 1, "SAME")
    x = batchnorm(x, tst, name="final_bn1")
    x = activation(x, name='final_act')
    
    x = tf.reduce_mean(x, [1,2]) #b x7 x 7 x 50 ==> b x 1 x 1 x 50==>b x 50 ==> b x 50
    W = tf.get_variable("FinalW", [n_classes, n_classes], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable("FinalB", [n_classes], initializer=tf.truncated_normal_initializer())
    
    out = tf.matmul(x, W) + b
                            

    return out

In [0]:
batch_size = 64
display_step = 80
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-3              # Learning rate start
tst = tf.placeholder(tf.bool)

In [11]:
# Setup the tensorflow...
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

print("Preparing the training & validation data...")
train_data, train_labels, filelist1, glen1 = setup_inputs(sess, "train.txt", "", batch_size=batch_size)
val_data, val_labels, filelist2, tlen1 = setup_inputs(sess, "val.txt", "",isTest=True, batch_size=batch_size)

max_iter = glen1*400
print("Preparing the training model with learning rate = %.5f..." % (lr))


with tf.variable_scope("ResNet") as scope:
  pred = ResNet(train_data, True)
  scope.reuse_variables()
  valpred = ResNet(val_data, False)

with tf.name_scope('Loss_and_Accuracy'):
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  with tf.control_dependencies(update_ops):
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=pred)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
  correct_prediction = tf.equal(tf.argmax(pred, 1), train_labels)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  
  correct_prediction2 = tf.equal(tf.argmax(valpred,1), val_labels)
  accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
  
  tf.summary.scalar('Loss', cost)
  tf.summary.scalar('Training_Accuracy', accuracy)

Preparing the training & validation data...


W0718 07:19:34.450525 139794417055616 deprecation.py:323] From <ipython-input-7-4690a01bcc50>:53: slice_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0718 07:19:34.464365 139794417055616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/input.py:374: range_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0718 07:19:34.493399 139794417055616 deprecation.py:3

Preparing the training model with learning rate = 0.00100...


W0718 07:19:35.551747 139794417055616 deprecation.py:323] From <ipython-input-8-a437f7771a6d>:19: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


Layer [1]  [64, 73, 73, 64]
[L-1] Build 0th residual block 0 with 64 channels
Layer [3]  [64, 73, 73, 64]
[L-2] Build 0th residual block 1 with 64 channels
Layer [5]  [64, 73, 73, 64]
[L-3] Build 0th residual block 2 with 64 channels
[L-4] Build 0th connection layer 2 from 64 to 128 channels
Layer [7]  [64, 36, 36, 128]
[L-5] Build 1th residual block 0 with 128 channels
Layer [9]  [64, 36, 36, 128]
[L-6] Build 1th residual block 1 with 128 channels
Layer [11]  [64, 36, 36, 128]
[L-7] Build 1th residual block 2 with 128 channels
Layer [13]  [64, 36, 36, 128]
[L-8] Build 1th residual block 3 with 128 channels
[L-9] Build 1th connection layer 3 from 128 to 256 channels
Layer [15]  [64, 17, 17, 256]
[L-10] Build 2th residual block 0 with 256 channels
Layer [17]  [64, 17, 17, 256]
[L-11] Build 2th residual block 1 with 256 channels
Layer [19]  [64, 17, 17, 256]
[L-12] Build 2th residual block 2 with 256 channels
Layer [21]  [64, 17, 17, 256]
[L-13] Build 2th residual block 3 with 256 channe

W0718 07:19:39.386953 139794417055616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[L-17] Build 3th residual block 0 with 512 channels
Layer [61]  [64, 8, 8, 512]
[L-18] Build 3th residual block 1 with 512 channels
Layer [63]  [64, 8, 8, 512]
[L-19] Build 3th residual block 2 with 512 channels


In [0]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
sess.run(init)
step = 0
writer = tf.summary.FileWriter("/tmp/log2", sess.graph)
summaries = tf.summary.merge_all()

print("We are going to train the ImageNet model based on ResNet!!!")
while (step * batch_size) < max_iter:
    epoch1=np.floor((step*batch_size)/glen1)
    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-3) & (epoch1 >2)):
        lr /= 10

    sess.run(optimizer,  feed_dict={learning_rate: lr, tst: True})

    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "tf_resnet_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step % display_step == 1:
        # calculate the loss
        
        loss, acc, summaries_string = sess.run([cost, accuracy, summaries], feed_dict={ tst: True})
        print("Iter=%d/epoch=%d, Loss=%.6f, Training Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1 ,loss, acc, lr))
        writer.add_summary(summaries_string, step)
        
#         if step*batch_size==82048:
#         import pdb
#         pdb.set_trace()

  
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "tf_resnet_model.ckpt")
print("Model saved in file: %s" % save_path)

We are going to train the ImageNet model based on ResNet!!!
Iter=64/epoch=0, Loss=9.193748, Training Accuracy=0.000000, lr=0.001000
Iter=5184/epoch=0, Loss=4.413558, Training Accuracy=0.078125, lr=0.001000
Iter=10304/epoch=0, Loss=3.996816, Training Accuracy=0.062500, lr=0.001000
Iter=15424/epoch=0, Loss=3.616088, Training Accuracy=0.093750, lr=0.001000
Iter=20544/epoch=0, Loss=3.696099, Training Accuracy=0.078125, lr=0.001000
Iter=25664/epoch=0, Loss=3.596581, Training Accuracy=0.078125, lr=0.001000
Iter=30784/epoch=0, Loss=3.571868, Training Accuracy=0.078125, lr=0.001000
Iter=35904/epoch=0, Loss=3.510183, Training Accuracy=0.062500, lr=0.001000
Iter=41024/epoch=0, Loss=3.360505, Training Accuracy=0.187500, lr=0.001000
Iter=46144/epoch=0, Loss=3.305911, Training Accuracy=0.187500, lr=0.001000
Iter=51264/epoch=0, Loss=3.256169, Training Accuracy=0.156250, lr=0.001000
Iter=56384/epoch=0, Loss=3.407115, Training Accuracy=0.171875, lr=0.001000
Iter=61504/epoch=0, Loss=3.240347, Training 

In [0]:
exit()